# Value Iteration Q-Learning Agent to play the Frozen Lake Game

*Winter is here. You and your friends were tossing around a frisbee at the park when you made a wild throw that left the frisbee out in the middle of the lake. The water is mostly frozen, but there are a few holes where the ice has melted. If you step into one of those holes, you'll fall into the freezing water. At this time, there's an international frisbee shortage, so it's absolutely imperative that you navigate across the lake and retrieve the disc. However, the ice is slippery, so you won't always move in the direction you intend. The surface is described using a grid like the following:*

<center> SFFF </center> 
<center>FHFH </center>
<center>FFFH </center>
<center>HFFG </center>

This grid is our environment where S is the agent's starting point, and it's safe. F represents the frozen surface and is also safe. H represents a hole, and if our agent steps in a hole in the middle of a frozen lake, well, that's not good. Finally, G represents the goal, which is the space on the grid where the prized frisbee is located.

The agent can navigate left, right, up, and down, and the episode ends when the agent reaches the goal or falls in a hole. It receives a reward of one if it reaches the goal, and zero otherwise.

- S: Starting point, Safe. 0 Reward \\
- F: Frozen Lake, Safe. 0 Reward \\
- H: Hole, Unsafe. Episode Ends. 0 Reward \\
- G: Goal. Episode ends. 1 Reward \\


In [1]:
import numpy as np
import gym
import random
import time
from IPython.display import clear_output
import matplotlib.pyplot as plt

## Creating the environment
Using open AI gym, we can load the environment of the Frozen Lake and render it using the following code:

In [2]:
env = gym.make("FrozenLake-v0")
env.render()


SFFF
FHFH
FFFH
HFFG


## Constructing the Q Table
We're now going to construct our Q-table, and initialize all the Q-values to zero for each state-action pair.

Remember, the number of rows in the table is equivalent to the size of the state space in the environment, and the number of columns is equivalent to the size of the action space. We can get this information using using env.observation_space.n and env.action_space.n, as shown below.

In [3]:
action_space_size = env.action_space.n
state_space_size = env.observation_space.n

q_table = np.zeros((state_space_size, action_space_size)) #Initialize q_table to a np array of zeros of shape state_space_size x action_space_size
print(q_table)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


## Initialize Q-Learning Parameters

Initialize the parameters for the q learning agent. Challenge: Change these values to get better results than the ones with these original values.

In [7]:
num_episodes = 10000 
max_steps_per_episode = 100 #Maximum number of moves per episode

learning_rate = 0.024 #Learning rate for updating the q values.
discount_rate = 0.999

exploration_rate = 1 #Initial exploration rate
max_exploration_rate = 1
min_exploration_rate = 0.01
exploration_decay_rate = 0.001 

## Train the Q-Learning agent

In [8]:
rewards_all_episodes = [] #This will store the rewards in each episode
for episode in range(num_episodes): #Iterate over each episode
    state = env.reset() #Reset the environment before starting each episode
    done = False #The done variable tells if the episode is over yet
    rewards_current_episode = 0
    for step in range(max_steps_per_episode): #Iterate over the steps

    ##### Exploration-exploitation trade-off #####
        #Generate a random sample between 0 and 1, explore if this value is less than the exploration rate, exploit otherwise
        exploration_rate_threshold = random.uniform(0,1) 
        if exploration_rate_threshold > exploration_rate:
            action = np.argmax(q_table[state,:])
        else:
            action = env.action_space.sample() #Hint: You want your action to be randomly chosen from {0,1,2,3} in this case
            
        new_state, reward, done, info = env.step(action) #Get the updated environment
        
        q_table[state, action] = (q_table[state, action] * (1 - learning_rate)) + learning_rate * (reward + discount_rate * np.max(q_table[new_state, :]))  #using formula for qnew
        state = new_state
        rewards_current_episode += reward 
        if done == True: 
            break
          
    #Update the exploration rate        
    exploration_rate = min_exploration_rate + (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate*episode)   #exponential decay of exploration_rate
    rewards_all_episodes.append(rewards_current_episode) 

Look at the values of the reward per thousand episodes. It should be increasing if your agent is learning properly.

In [9]:
rewards_per_thousand_episodes = np.split(np.array(rewards_all_episodes),num_episodes/1000)
count = 1000

print("********Average reward per thousand episodes********\n")
for r in rewards_per_thousand_episodes:
    print(count, ": ", str(sum(r/1000)))
    count += 1000

********Average reward per thousand episodes********

1000 :  0.03900000000000003
2000 :  0.1520000000000001
3000 :  0.32100000000000023
4000 :  0.5440000000000004
5000 :  0.6630000000000005
6000 :  0.6720000000000005
7000 :  0.6750000000000005
8000 :  0.7060000000000005
9000 :  0.7030000000000005
10000 :  0.6920000000000005


In [10]:
#Final Q Table
print(q_table)

[[0.77480665 0.72410626 0.72578837 0.72309978]
 [0.23034568 0.22612753 0.15597664 0.66139691]
 [0.53814728 0.29007164 0.22698974 0.25107615]
 [0.18134754 0.02865943 0.01747631 0.04441663]
 [0.77613482 0.53713512 0.56196641 0.53829089]
 [0.         0.         0.         0.        ]
 [0.26629804 0.14182807 0.41391491 0.09070511]
 [0.         0.         0.         0.        ]
 [0.49524967 0.51211565 0.53996455 0.77807953]
 [0.55263998 0.77700649 0.49519815 0.40145525]
 [0.66955616 0.48353622 0.32157349 0.31806089]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.5544824  0.62992809 0.84771408 0.54336761]
 [0.73652761 0.93551273 0.80251521 0.81622528]
 [0.         0.         0.         0.        ]]


## Watch the trained agent play Frozen Lake

In [ ]:
for episode in range(10):
    state = env.reset()
    done = False
    print("*****EPISODE ", episode+1, "*****\n\n\n\n")
    time.sleep(1)
    for step in range(max_steps_per_episode):        
        clear_output(wait=True)
        env.render()
        time.sleep(0.3)
        action = np.argmax(q_table[state,:])  #Since this is not part of training, we want the agent to exploit all the time.     
        new_state, reward, done, info = env.step(action)
        if done:
            clear_output(wait=True)
            env.render()
            if reward == 1:
                print("****You reached the goal!****")
                time.sleep(3)
            else:
                print("****You fell through a hole!****")
                time.sleep(3)
                clear_output(wait=True)
            break
        state = new_state
env.close()

  (Left)
SFFF
FHFH
FFFH
HFFG
